In [18]:
import os

# Option 1 (Recommended): Test environment variable reading
api_key_test_env = os.getenv("OPENROUTER_API_KEY")
print(f"API Key (from env var): '{api_key_test_env}'")

# Option 2 (For temporary direct assignment if env var fails):
api_key_test_direct = "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" # PASTE YOUR ACTUAL KEY HERE
print(f"API Key (direct assignment): '{api_key_test_direct}'")

# Then, verify length (should be around 64-68 characters for OpenRouter v1 keys)
if api_key_test_env: # Or api_key_test_direct if you uncomment that line
    print(f"API Key length: {len(api_key_test_env)}")
else:
    print("API Key is None or empty.")

API Key (from env var): 'None'
API Key (direct assignment): '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'
API Key is None or empty.


In [19]:
# Stap 0: Initialisatie en Configuratie (Aangepast voor OpenRouter)

import pdfplumber
import re
import requests
import os
import time
from PIL import Image
import pytesseract
from pdf2image import convert_from_path

# Configureer je OpenRouter API sleutel.
# Optie 2: Hardcode de sleutel hier (MINDER VEILIG, ALLEEN VOOR TESTEN!)
OPENROUTER_API_KEY = "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" # <-- JE NIEUWE, GELDIGE KEY # <-- DIRECT DE KEY HIER INVULLEN

# Verwijder of commentarieer de volgende regels als je hardcodet:
# if not OPENROUTER_API_KEY:
#     raise ValueError("OpenRouter API Key is niet ingesteld. Stel 'OPENROUTER_API_KEY' in als omgevingsvariabele.")

# OpenRouter API URL (dit is de algemene URL voor OpenRouter's chat completions)
OPENROUTER_API_URL = "https://openrouter.ai/api/v1/chat/completions"

# Het SPECIFIEKE DeepSeek model dat je via OpenRouter wilt gebruiken
MODEL_NAME = "deepseek/deepseek-r1:free"

print("Configuratie geladen voor OpenRouter.")

Configuratie geladen voor OpenRouter.


In [20]:
# Stap 1: Definieer de lijst van te verwerken PDF-bestanden

# Maak hier een lijst van alle PDF-paden die je wilt analyseren.
# Voeg zoveel paden toe als je nodig hebt.
pdf_file_paths = [
    "/Users/mohamedelharchaoui/Downloads/Assignment 3/0083_01-01-2025 tot 01-07-2026_akkoord.pdf",
    "/Users/mohamedelharchaoui/Downloads/Assignment 3/0182_01-01-2025 tot 01-01-2027_akkoord.pdf",
    "/Users/mohamedelharchaoui/Downloads/Assignment 3/1069_01-01-2025 tot 01-01-2027_akkoord.pdf",
    "/Users/mohamedelharchaoui/Downloads/Assignment 3/2070_01-01-2025 tot 01-01-2028_akkoord.pdf"
    # Voeg hier meer paden toe, bijvoorbeeld:
    # "/pad/naar/jouw/andere_cao.pdf"
]

print(f"{len(pdf_file_paths)} PDF-bestanden gedefinieerd voor verwerking.")

# --- Functiedefinities voor text-extractie ---

def extract_text_from_pdf_with_ocr(pdf_path, min_text_length=100):
    """
    Probeert eerst tekst digitaal te extraheren. Als dat te weinig tekst oplevert,
    schakelt het automatisch over naar OCR met Tesseract.
    """
    text = ""
    # Eerste poging: digitale extractie met pdfplumber
    try:
        with pdfplumber.open(pdf_path) as pdf:
            for page in pdf.pages:
                page_text = page.extract_text()
                if page_text:
                    text += page_text + "\n"
        print("Methode: Digitale extractie (pdfplumber) geslaagd.")
    except Exception as e:
        print(f"Fout tijdens digitale extractie: {e}. Probeert nu OCR.")
        text = "" # Reset de tekst voor de OCR poging

    # Controleer of de digitale extractie genoeg tekst heeft opgeleverd
    if len(text.strip()) >= min_text_length:
        return text

    # Tweede poging: OCR met Tesseract voor ingescande PDF's
    print(f"Digitale extractie leverde < {min_text_length} tekens op. Overschakelen naar OCR...")
    try:
        # Zet de PDF-pagina's om naar afbeeldingen
        images = convert_from_path(pdf_path)
        ocr_text = ""
        for i, image in enumerate(images):
            print(f"  Verwerken van pagina {i+1} met OCR...")
            # Gebruik Tesseract om tekst van de afbeelding te lezen (specificeer de Nederlandse taal)
            ocr_text += pytesseract.image_to_string(image, lang='nld') + "\n"
        print("Methode: OCR-extractie (Tesseract) geslaagd.")
        return ocr_text
    except Exception as e:
        print(f"Fout tijdens OCR-extractie: {e}")
        return None

4 PDF-bestanden gedefinieerd voor verwerking.


In [21]:
# Stap 2: Gebruik van regex om zinnen met percentages te extraheren

def extract_percentage_sentences(text):
    """
    Zoekt naar zinnen in de tekst die percentages bevatten,
    met een focus op contexten die gerelateerd kunnen zijn aan loonstijgingen.
    """
    if not text:
        return []

    # Gecorrigeerd regex patroon. '[^.?!]' wordt gebruikt om de hele zin te vangen.
    percentage_pattern = re.compile(
        r'([^.?!]*?(?:loon|salaris|cao|verhoging|stijging|toeslag)\s*[^.?!]*?\d[\d.,]*\s?%\s*[^.?!]*?[.?!])',
        re.IGNORECASE
    )
    sentences_with_percentage = percentage_pattern.findall(text)

    # Verwijder overtollige spaties en vervang de ECHTE newline karakters ('\n')
    clean_sentences = [s.strip().replace('\n', ' ') for s in sentences_with_percentage]
    return clean_sentences

In [22]:
import json
from json import JSONDecoder

def classify_with_deepseek(sentence, api_key, api_url, model_name, max_retries=3, delay=5):
    """
    Roept de API aan via OpenRouter om een zin te classificeren en om uitleg te vragen.
    Is nu in staat om MEERDERE loonstijgingen uit één zin te extraheren en is robuust tegen JSON-fouten.
    """
    if not api_key:
        print("Fout: OpenRouter API-sleutel is leeg of niet ingesteld.")
        return None

    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json",
        "HTTP-Referer": "https://localhost/sakkal-cao-analyzer",
        "X-Title": "Team Sakkal CAO Analyzer"
    }
    
    # Finale prompt, nu met een extra voorbeeld om referentiedatums te negeren.
    system_prompt = """Je bent een expert in het analyseren van Nederlandse CAO-teksten. Je taak is om te bepalen of een zin concrete, definitieve loonstijgingen beschrijft en **alle** details daarvan te extraheren.

**Instructies:**
- Een zin kan **meerdere** loonstijgingen bevatten (bijv. in een opsomming). Je moet ze **allemaal** vinden.
- Classificeer alleen **daadwerkelijke, gegarandeerde afspraken** over salarisverhogingen als 'Loonstijging'. Dit zijn de 'normale' loonsverhogingen die voor iedereen gelden.
- **Negeer** zinnen die beginnen met 'Rekenvoorbeeld', 'Voorbeeld:', 'Stel dat', of 'Berekening'.
- **Negeer** structurele salarisverschillen tussen functies (bv. 'Level 2 is 10% hoger dan Level 1').
- **Negeer** verhogingen die een omzetting zijn van andere arbeidsvoorwaarden (zoals het omzetten van verlofdagen in salaris) of specifieke toeslagen/vergoedingen (zoals een reiskostenvergoeding).
- **Cruciaal: Negeer voorwaardelijke verhogingen** die afhankelijk zijn van een conditie (bv. 'tenzij', 'indien').
- **Cruciaal: Negeer vergelijkende berekeningen.** Als een stijging wordt beschreven 'ten opzichte van' een datum in het verleden, is dit een vergelijking, geen nieuwe afspraak.

**Jouw taak:**
Analyseer de zin en geef een JSON-object terug met DRIE sleutels:
1. 'classificatie': 'Loonstijging' of 'Geen Loonstijging'.
2. 'verhogingen': Een **lijst** van JSON-objecten. Elk object bevat 'datum' en 'percentage'. Als er geen gegarandeerde loonstijgingen zijn, moet deze lijst leeg zijn: `[]`.
3. 'uitleg': een korte, duidelijke toelichting op je keuze.

**BELANGRIJK:** Je antwoord MOET **uitsluitend** een enkel, geldig JSON-object zijn. Geen extra tekst."""

    # Gebruik van triple quotes (''') voor de JSON-strings om escaping-fouten te voorkomen.
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": "De salarissen stijgen met 2% op 01-01-2025 en met nog eens 3% op 01-07-2025."},
        {"role": "assistant", "content": '''{"classificatie": "Loonstijging", "verhogingen": [{"datum": "01/01/2025", "percentage": 2.0}, {"datum": "01/07/2025", "percentage": 3.0}], "uitleg": "De zin bevat twee concrete loonstijgingen."}'''},
        {"role": "user", "content": "Met ingang van 1 januari 2025 worden deze verlofdagen omgezet in een structurele salarisverhoging van 0,84% tenzij werkgever deze verlofdagen al eerder heeft omgezet."},
        {"role": "assistant", "content": '''{"classificatie": "Geen Loonstijging", "verhogingen": [], "uitleg": "De verhoging is voorwaardelijk en een omzetting van een andere arbeidsvoorwaarde, dus het is geen standaard loonstijging."}'''},
        {"role": "user", "content": "het eindloon stijgt ten opzichte van het eindloon op 31 december 2024 met 6,9%."},
        {"role": "assistant", "content": '''{"classificatie": "Geen Loonstijging", "verhogingen": [], "uitleg": "Dit is een vergelijkende berekening ten opzichte van een referentiedatum, geen nieuwe loonafspraak met een ingangsdatum."}'''},
        # NIEUW VOORBEELD om de 'berekend door' structuur te leren
        {"role": "user", "content": "Het minimum bruto uursalaris van de Klantadviseur Level 1 per 1 januari 2026 wordt berekend door op het wettelijk minimumloon van 1 januari 2026 een percentage van 3,85% te tellen."},
        {"role": "assistant", "content": '''{"classificatie": "Loonstijging", "verhogingen": [{"datum": "01/01/2026", "percentage": 3.85}], "uitleg": "De zin beschrijft een gegarandeerde verhoging als een percentage bovenop het wettelijk minimumloon, dit is een geldige loonstijging."}'''},
        {"role": "user", "content": sentence}
    ]

    data = { "model": model_name, "messages": messages, "response_format": { "type": "json_object" }, "max_tokens": 400, "temperature": 0.0 }
    
    for attempt in range(max_retries):
        try:
            response = requests.post(api_url, json=data, headers=headers, timeout=45)
            response.raise_for_status()

            # Robuuste JSON-parsing die extra data na een geldig object negeert
            try:
                content = response.text
                full_response_json = json.loads(content)
                message_content = full_response_json['choices'][0]['message']['content']
                
                # --- FINALE ROBUUSTE PARSING (VERBETERD) ---
                start_index = message_content.find('{')
                end_index = message_content.rfind('}')
                
                if start_index != -1 and end_index != -1 and end_index > start_index:
                    json_str = message_content[start_index : end_index + 1]
                    try:
                        return json.loads(json_str)
                    except json.JSONDecodeError as e:
                        print(f"Fout bij parsen van geïsoleerd JSON-object: {e}")
                        print(f"Onbewerkte content van AI: {message_content}")
                        return None
                else:
                    print("Geen JSON-object gevonden in de AI-respons.")
                    print(f"Onbewerkte content van AI: {message_content}")
                    return None
                # --- EINDE FINALE PARSING ---

            except (json.JSONDecodeError, KeyError, IndexError, TypeError) as e:
                print(f"Fout bij parsen van de volledige API-respons: {e}")
                print(f"Ontvangen tekst: {response.text}")
                return None
        except requests.exceptions.RequestException as e:
            print(f"Netwerkfout: {e}")
            if attempt < max_retries - 1:
                time.sleep(delay)
            else:
                return None
            
    print(f"Fout na {max_retries} pogingen.")
    return None

In [35]:
# Stap 4: Hoofd-loop voor het verwerken van alle gedefinieerde PDF's

from datetime import datetime
import json

# Dictionary om de eindresultaten van alle PDF's op te slaan
final_json_output = {}

# Loop door elk opgegeven PDF-pad
for pdf_path in pdf_file_paths:
    pdf_filename = os.path.basename(pdf_path)
    print("\n" + "="*80)
    print(f"--- Begin analyse van: {pdf_filename} ---\n")
    
    # Stap 1: Tekst extractie
    extracted_text = extract_text_from_pdf_with_ocr(pdf_path)
    if not extracted_text:
        print(f"Kon geen tekst extraheren uit {pdf_filename}. Bestand wordt overgeslagen.")
        final_json_output[pdf_filename] = {"error": "Tekstextractie mislukt", "verhogingen": []}
        continue

    # Stap 2: Zinnen met percentages vinden
    sentences = extract_percentage_sentences(extracted_text)
    if not sentences:
        print(f"Geen relevante zinnen met percentages gevonden in {pdf_filename}.")
        final_json_output[pdf_filename] = {"error": "Geen relevante zinnen gevonden", "verhogingen": []}
        continue

    print(f"{len(sentences)} zinnen gevonden voor analyse.")
    
    # Stap 3: Classificatie en verzameling
    all_found_increases = []
    print("\nBeginnen met classificatie via DeepSeek API:")
    for i, sentence in enumerate(sentences):
        short_sentence = (sentence[:120] + '...') if len(sentence) > 120 else sentence
        print(f"\nVerwerken zin {i+1}/{len(sentences)}: {short_sentence}")
        
        result_json = classify_with_deepseek(sentence, OPENROUTER_API_KEY, OPENROUTER_API_URL, MODEL_NAME)
        
        if result_json:
            classificatie = result_json.get('classificatie', 'Onbekend')
            verhogingen = result_json.get('verhogingen', [])
            uitleg = result_json.get('uitleg', '')
            
            if classificatie == 'Loonstijging' and verhogingen:
                print(f"  Resultaat: ✅ {len(verhogingen)} loonstijging(en) gevonden.")
                all_found_increases.extend(verhogingen)
            else:
                print(f"  Resultaat: ❌ Geen Loonstijging")
        else:
            print("  Resultaat: ❓ Fout bij het classificeren van de zin.")
        
        time.sleep(1)

    # Stap 4: Resultaten voor de huidige PDF groeperen en aggregeren
    grouped_increases = {}
    for increase in all_found_increases:
        datum = increase.get('datum')
        percentage = increase.get('percentage')
        if datum and percentage and datum != 'N.v.t.' and percentage != 'N.v.t.':
            if datum not in grouped_increases:
                grouped_increases[datum] = []
            if isinstance(percentage, (int, float)):
                grouped_increases[datum].append(percentage)

    aggregated_increases = []
    for datum, percentages in grouped_increases.items():
        # Sorteer percentages om de output voorspelbaar te maken
        percentages.sort()
        # Gebruik de samengevoegde string-notatie die we eerder hadden
        percentage_str = "/".join([f"{p:.2f}".replace('.', ',') + '%' for p in percentages])
        aggregated_increases.append({'datum': datum, 'percentage': percentage_str})
    
    # Sorteer de uiteindelijke lijst op datum
    def sort_key(item):
        try:
            return datetime.strptime(item.get('datum', ''), '%d/%m/%Y')
        except (ValueError, TypeError):
            return datetime.max
    aggregated_increases.sort(key=sort_key)
    
    # Voeg het resultaat toe aan de hoofd-dictionary
    final_json_output[pdf_filename] = {"verhogingen": aggregated_increases}
    print(f"\nAnalyse voor {pdf_filename} voltooid.")

# --- Aan het einde, na de loop, print de volledige JSON-output ---
print("\n" + "="*80)
print("--- Alle PDF's zijn verwerkt. Hier is de volledige JSON-output: ---")
print("="*80)
print(json.dumps(final_json_output, indent=2, ensure_ascii=False))


--- Begin analyse van: 0083_01-01-2025 tot 01-07-2026_akkoord.pdf ---

Methode: Digitale extractie (pdfplumber) geslaagd.
Geen relevante zinnen met percentages gevonden in 0083_01-01-2025 tot 01-07-2026_akkoord.pdf.

--- Begin analyse van: 0182_01-01-2025 tot 01-01-2027_akkoord.pdf ---

Methode: Digitale extractie (pdfplumber) geslaagd.
Geen relevante zinnen met percentages gevonden in 0182_01-01-2025 tot 01-01-2027_akkoord.pdf.

--- Begin analyse van: 1069_01-01-2025 tot 01-01-2027_akkoord.pdf ---

Methode: Digitale extractie (pdfplumber) geslaagd.
Digitale extractie leverde < 100 tekens op. Overschakelen naar OCR...
  Verwerken van pagina 1 met OCR...
  Verwerken van pagina 2 met OCR...
  Verwerken van pagina 3 met OCR...
  Verwerken van pagina 4 met OCR...
  Verwerken van pagina 5 met OCR...
  Verwerken van pagina 6 met OCR...
  Verwerken van pagina 7 met OCR...
  Verwerken van pagina 8 met OCR...
Methode: OCR-extractie (Tesseract) geslaagd.
Geen relevante zinnen met percentages ge

In [ ]:
# --- STAP 4: DEFINITIEVE BACKEND-SERVER IN ÉÉN CEL (Versie 3.7 - Strikte AI) ---
# Deze cel bevat de definitieve, werkende code met de strengere AI-instructies 
# om rekenvoorbeelden te negeren.

# --- Imports ---
import os
import re
import json
import time
import uuid
import traceback
from datetime import datetime

import pandas as pd
import pdfplumber
import pytesseract
import requests
from flask import Flask, request, jsonify, send_file
from flask_cors import CORS
from pdf2image import convert_from_path
from openpyxl.styles import Font
from openpyxl.cell.rich_text import CellRichText, TextBlock
from openpyxl.cell.text import InlineFont 
from openpyxl.utils import get_column_letter
from werkzeug.utils import secure_filename
try:
    from PIL import Image
except ImportError:
    import Image


# --- Configuratie ---
OPENROUTER_API_KEY = "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"
OPENROUTER_API_URL = "https://openrouter.ai/api/v1/chat/completions"
MODEL_NAME = "deepseek/deepseek-r1:free"


# --- HULPFUNCTIE: PDF Extractie ---
def extract_text_from_pdf_with_ocr(pdf_path, min_text_length=100):
    text = ""
    try:
        with pdfplumber.open(pdf_path) as pdf:
            for page in pdf.pages:
                page_text = page.extract_text()
                if page_text: text += page_text + "\n"
        print(f"Methode: Digitale extractie geslaagd voor {os.path.basename(pdf_path)}.")
    except Exception as e:
        print(f"Fout tijdens digitale extractie: {e}. Probeert nu OCR.")
        text = ""

    if len(text.strip()) >= min_text_length: return text

    print(f"Digitale extractie leverde te weinig tekst op. Overschakelen naar OCR...")
    try:
        images = convert_from_path(pdf_path)
        ocr_text = ""
        for i, image in enumerate(images):
            print(f"  Verwerken van pagina {i+1} met OCR...")
            ocr_text += pytesseract.image_to_string(image, lang='nld') + "\n"
        print("Methode: OCR-extractie (Tesseract) geslaagd.")
        return ocr_text
    except Exception as e:
        print(f"Fout tijdens OCR-extractie: {e}")
        return None

# --- HULPFUNCTIE: Zinnen vinden (Stabiele versie) ---
def extract_percentage_sentences(text):
    if not text:
        return []

    text = text.replace('\n', ' ').replace('\r', ' ')
    text = re.sub(r'\s+', ' ', text)
    sentences = re.split(r'(?<=[.?!])\s+', text)
    
    keywords = ['loon', 'salaris', 'cao', 'verhoging', 'stijging', 'toeslag', 'schaalsalarissen']
    percentage_pattern = re.compile(r'\d[\d.,]*\s?%')
    
    relevant_sentences = []
    for sentence in sentences:
        if not sentence: continue
        sentence_lower = sentence.lower()
        has_keyword = any(keyword in sentence_lower for keyword in keywords)
        has_percentage = percentage_pattern.search(sentence_lower)
        
        if has_keyword and has_percentage:
            relevant_sentences.append(sentence.strip())
            
    return relevant_sentences


# --- HULPFUNCTIE: AI Classificatie (Striktere Prompt) ---
def classify_with_deepseek(sentence, api_key, api_url, model_name, max_retries=3, delay=2):
    headers = {
        "Authorization": f"Bearer {api_key}", "Content-Type": "application/json",
        "HTTP-Referer": "https://localhost/sakkal-cao-analyzer", "X-Title": "Team Sakkal CAO Analyzer"
    }
    system_prompt = """Je bent een expert in het analyseren van Nederlandse CAO-teksten. Je taak is om **alleen** concrete, definitieve loonstijgingen te vinden, te extraheren en te categoriseren.

**ZEER BELANGRIJKE REGELS OM TE VOLGEN:**
- **NEGEER VOLLEDIG** alle zinnen die onderdeel zijn van een voorbeeld, berekening of hypothese.
- Zoek naar sleutelwoorden zoals: **"Voorbeeld:", "Rekenvoorbeeld", "Stel dat", "Als ... dan", "Berekening"**. Als je zo'n sleutelwoord ziet, is de zin bijna altijd een voorbeeld en moet je een lege `verhogingen` lijst teruggeven.
- Negeer ook verhogingen die voorwaardelijk zijn ('tenzij', 'indien') of die alleen een vergelijking maken met het verleden.

**Jouw taak:**
Analyseer de zin en geef een JSON-object terug. Het object moet een lijst `verhogingen` bevatten. Voor **elke** gevonden loonstijging, maak een object met VIER sleutels:
1. `datum`: De ingangsdatum (formaat: "DD/MM/YYYY").
2. `percentage`: Het percentage (als een getal, bv. 3.5).
3. `categorie`: Classificeer het type verhoging. Kies uit: "standaard", "verlofdag_omzetting", "dienstjaren_toeslag", "WML_koppeling", "anders".
4. `uitleg`: Een korte toelichting op je keuze.

**BELANGRIJK:** Je antwoord MOET **uitsluitend** een enkel, geldig JSON-object zijn met de sleutel 'verhogingen'. Geen extra tekst."""
    messages = [
        {"role": "system", "content": system_prompt},
        # Goede voorbeelden
        {"role": "user", "content": "De salarissen stijgen met 2% op 01-01-2025 en met nog eens 3% op 01-07-2025."},
        {"role": "assistant", "content": '''{"verhogingen": [{"datum": "01/01/2025", "percentage": 2.0, "categorie": "standaard", "uitleg": "Een standaard collectieve verhoging."}, {"datum": "01/07/2025", "percentage": 3.0, "categorie": "standaard", "uitleg": "Een tweede standaard collectieve verhoging."}]}'''},
        {"role": "user", "content": "Het minimumloon wordt per 1/1/26 berekend door 3,85% bovenop het WML te tellen."},
        {"role": "assistant", "content": '''{"verhogingen": [{"datum": "01/01/2026", "percentage": 3.85, "categorie": "WML_koppeling", "uitleg": "De verhoging is direct gekoppeld aan het WML."}]}'''},
        
        # Slechte voorbeelden (rekenvoorbeelden)
        {"role": "user", "content": "Voorbeeld: Als het bruto uursalaris volgens de cao op 1 januari 2026 met 3,88% stijgt, dan zal het uursalaris..."},
        {"role": "assistant", "content": '''{"verhogingen": [], "uitleg": "De zin begint met 'Voorbeeld:' en bevat 'Als...dan', wat duidt op een hypothetische berekening."}'''},
        {"role": "user", "content": "Rekenvoorbeeld: Een medewerker met een salaris van € 3.000,00 ontvangt per 1 januari 2026 een verhoging van 3,88%."},
        {"role": "assistant", "content": '''{"verhogingen": [], "uitleg": "De zin begint met 'Rekenvoorbeeld' en is dus een voorbeeld, geen afspraak."}'''},
        {"role": "user", "content": sentence}
    ]
    data = { "model": MODEL_NAME, "messages": messages, "response_format": { "type": "json_object" }, "max_tokens": 500, "temperature": 0.0 }
    
    for attempt in range(max_retries):
        try:
            response = requests.post(api_url, json=data, headers=headers, timeout=45)
            response.raise_for_status()
            full_response_json = response.json()
            message_content_str = full_response_json['choices'][0]['message']['content']
            json_start = message_content_str.find('{')
            json_end = message_content_str.rfind('}')
            if json_start != -1 and json_end != -1:
                return json.loads(message_content_str[json_start:json_end+1])
            return None
        except Exception as e:
            print(f"Fout in classify_with_deepseek: {e}")
            if attempt < max_retries - 1: time.sleep(delay)
    return None

# --- HULPFUNCTIE: PDF Analyse Orchestratie ---
def analyze_pdfs(pdf_file_paths):
    final_results = {}
    for pdf_path in pdf_file_paths:
        filename = os.path.basename(pdf_path)
        print(f'--- Start analyse voor: {filename} ---')
        text = extract_text_from_pdf_with_ocr(pdf_path)
        sentences = extract_percentage_sentences(text) if text else []
        if not sentences:
            final_results[filename] = {"error": "Geen zinnen gevonden", "verhogingen": []}
            print(f"  -> ❌ Geen relevante zinnen gevonden in {filename}.")
            continue
        
        increases = []
        print(f"  -> ✅ {len(sentences)} potentieel relevante zin(nen) gevonden.")
        for i, sentence in enumerate(sentences):
            print(f"  Verwerken zin {i+1}/{len(sentences)}...")
            res = classify_with_deepseek(sentence, OPENROUTER_API_KEY, OPENROUTER_API_URL, MODEL_NAME)
            if res and 'verhogingen' in res and isinstance(res['verhogingen'], list) and len(res['verhogingen']) > 0:
                print(f"    -> ✅ {len(res['verhogingen'])} verhoging(en) gevonden in zin.")
                increases.extend(res['verhogingen'])
            else:
                print("    -> ❌ Geen concrete verhoging geclassificeerd in zin.")
            time.sleep(1.5)
        
        final_results[filename] = {"verhogingen": increases}
        print(f'--- Analyse voor {filename} voltooid ---')
    return final_results

# --- HULPFUNCTIE: Excel Generatie (Stabiele versie) ---
def create_excel_summary(analysis_results, output_filename):
    COLOR_MAP = {
        "verlofdag_omzetting": "FFC000", "dienstjaren_toeslag": "0070C0",
        "WML_koppeling": "00B050", "anders": "7030A0", "standaard": "000000"
    }
    
    RICH_TEXT_FONT_MAP = {cat: InlineFont(color=color) for cat, color in COLOR_MAP.items()}
    CELL_STYLE_FONT_MAP = {cat: Font(color=color) for cat, color in COLOR_MAP.items()}
    DEFAULT_RICH_TEXT_FONT = RICH_TEXT_FONT_MAP["standaard"]

    LEGEND = {
        "standaard": "Standaard loonsverhoging.", "WML_koppeling": "Gekoppeld aan WML.",
        "verlofdag_omzetting": "Omzetting van verlofdagen.", "dienstjaren_toeslag": "O.b.v. dienstjaren.",
        "anders": "Andere specifieke verhoging."
    }

    grouped_results, max_dates = {}, 0
    for filename, data in analysis_results.items():
        by_date = {}
        for inc in data.get('verhogingen', []):
            date_str = inc.get('datum')
            if not date_str or not isinstance(inc.get('percentage'), (int, float)): continue
            try:
                date_key = datetime.strptime(date_str, '%d/%m/%Y')
                if date_key not in by_date: by_date[date_key] = []
                by_date[date_key].append(inc)
            except (ValueError, TypeError): continue
        
        sorted_dates = sorted(by_date.keys())
        grouped_results[filename] = [(d.strftime('%d/%m/%Y'), by_date[d]) for d in sorted_dates]
        max_dates = max(max_dates, len(sorted_dates))

    processed_data = []
    for filename, date_groups in grouped_results.items():
        row = {'Bestandsnaam': filename}
        for i, (date, increases) in enumerate(date_groups):
            row[f'{i+1}e datum'] = date
            row[f'{i+1}e percentages'] = " / ".join([f"{inc.get('percentage', 0):.2f}%".replace('.', ',') for inc in increases])
        processed_data.append(row)

    df = pd.DataFrame()
    if processed_data:
        headers = ['Bestandsnaam'] + [item for i in range(1, max_dates + 1) for item in (f'{i}e datum', f'{i}e percentages')]
        df = pd.DataFrame(processed_data, columns=headers).fillna('')

    with pd.ExcelWriter(output_filename, engine='openpyxl') as writer:
        df.to_excel(writer, index=False, sheet_name='Samenvatting')
        ws = writer.sheets['Samenvatting']
        header_map = {cell.value: i + 1 for i, cell in enumerate(ws[1])}

        for row_idx, filename in enumerate(df['Bestandsnaam'], start=2):
            for i, (date, increases) in enumerate(grouped_results.get(filename, [])):
                col_idx = header_map.get(f'{i+1}e percentages')
                if not col_idx: continue
                
                cell = ws.cell(row=row_idx, column=col_idx)
                payload = []
                for j, inc in enumerate(increases):
                    if j > 0 and payload: payload.append(TextBlock(DEFAULT_RICH_TEXT_FONT, " / "))
                    p_str = f"{inc.get('percentage', 0):.2f}%".replace('.', ',')
                    font = RICH_TEXT_FONT_MAP.get(inc.get('categorie', 'standaard'), DEFAULT_RICH_TEXT_FONT)
                    payload.append(TextBlock(font, p_str))
                
                if payload: cell.value = CellRichText(payload)

        for i, col_cells in enumerate(ws.columns):
            length = max(len(str(cell.value)) if not isinstance(cell.value, CellRichText) else len("".join(b.text for b in cell.value)) for cell in col_cells)
            ws.column_dimensions[get_column_letter(i + 1)].width = length + 4
        if not df.empty:
            legend_col = len(df.columns) + 2
            ws.cell(row=1, column=legend_col, value="LEGENDA").font = Font(bold=True)
            for r, (cat, desc) in enumerate(LEGEND.items(), start=2):
                font_style = CELL_STYLE_FONT_MAP.get(cat, CELL_STYLE_FONT_MAP['standaard'])
                ws.cell(row=r, column=legend_col, value=desc).font = font_style
            ws.column_dimensions[get_column_letter(legend_col)].width = max(len(d) for d in LEGEND.values()) + 4

    print(f'Excel-bestand succesvol aangemaakt op: {output_filename}')


# --- Flask Webserver ---
app = Flask(__name__)
CORS(app)
UPLOAD_FOLDER = os.path.join(os.path.dirname(os.path.realpath('__file__')), 'temp_uploads')
os.makedirs(UPLOAD_FOLDER, exist_ok=True)

@app.route('/api/process', methods=['POST'])
def process_uploaded_pdfs():
    print("\n[SERVER LOG] Received new request for /api/process")
    if 'files' not in request.files: return jsonify({"error": "Geen bestanden meegegeven"}), 400
    files = request.files.getlist('files')
    if not files or files[0].filename == '': return jsonify({"error": "Geen bestanden geselecteerd"}), 400

    request_id = str(uuid.uuid4())
    request_upload_dir = os.path.join(UPLOAD_FOLDER, request_id)
    os.makedirs(request_upload_dir, exist_ok=True)

    saved_file_paths = []
    for f in files:
        path = os.path.join(request_upload_dir, secure_filename(f.filename))
        f.save(path)
        saved_file_paths.append(path)

    try:
        analysis_results = analyze_pdfs(saved_file_paths)
        if not analysis_results or all(not v.get('verhogingen') for v in analysis_results.values()):
            print("[FOUT] Analyse heeft geen bruikbare loonsverhogingen opgeleverd.")
            print("Analyse resultaten:", json.dumps(analysis_results, indent=2))
            return jsonify({"error": "Analyse heeft geen bruikbare loonsverhogingen opgeleverd."}), 500

        output_excel_path = os.path.join(request_upload_dir, f"cao_samenvatting_{request_id}.xlsx")
        create_excel_summary(analysis_results, output_excel_path)
        
        return send_file(output_excel_path, as_attachment=True, download_name='cao_samenvatting.xlsx', mimetype='application/vnd.openxmlformats-officedocument.spreadsheetml.sheet')
    except Exception as e:
        print(f"[FATALE FOUT] Onverwachte serverfout opgetreden in /api/process:")
        traceback.print_exc()
        return jsonify({"error": f"Onverwachte serverfout: {str(e)}"}), 500

# --- Server Start ---
print('--- Flask server starten vanaf notebook (v3.7 - Strikte AI) ---')
print(f'Uploads worden opgeslagen in: {os.path.abspath(UPLOAD_FOLDER)}')
print('Server is live op http://127.0.0.1:5001')
app.run(host='127.0.0.1', port=5001, debug=True, use_reloader=False)

--- Flask server starten vanaf notebook (v3.7 - Strikte AI) ---
Uploads worden opgeslagen in: /Users/mohamedelharchaoui/Downloads/Tweedejaarsproject/excel-pdf-automaton/src/backend/temp_uploads
Server is live op http://127.0.0.1:5001
 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5001
Press CTRL+C to quit



[SERVER LOG] Received new request for /api/process
--- Start analyse voor: 2070_01-01-2025_tot_01-01-2028_akkoord.pdf ---
Methode: Digitale extractie geslaagd voor 2070_01-01-2025_tot_01-01-2028_akkoord.pdf.
  -> ✅ 22 potentieel relevante zin(nen) gevonden.
  Verwerken zin 1/22...
    -> ✅ 1 verhoging(en) gevonden in zin.
  Verwerken zin 2/22...
    -> ✅ 1 verhoging(en) gevonden in zin.
  Verwerken zin 3/22...
    -> ✅ 1 verhoging(en) gevonden in zin.
  Verwerken zin 4/22...
    -> ✅ 1 verhoging(en) gevonden in zin.
  Verwerken zin 5/22...
    -> ❌ Geen concrete verhoging geclassificeerd in zin.
  Verwerken zin 6/22...
    -> ❌ Geen concrete verhoging geclassificeerd in zin.
  Verwerken zin 7/22...
    -> ❌ Geen concrete verhoging geclassificeerd in zin.
  Verwerken zin 8/22...
    -> ❌ Geen concrete verhoging geclassificeerd in zin.
  Verwerken zin 9/22...
    -> ❌ Geen concrete verhoging geclassificeerd in zin.
  Verwerken zin 10/22...
    -> ❌ Geen concrete verhoging geclassificeer

127.0.0.1 - - [29/Jun/2025 07:02:57] "POST /api/process HTTP/1.1" 200 -


--- Analyse voor 2070_01-01-2025_tot_01-01-2028_akkoord.pdf voltooid ---
Excel-bestand succesvol aangemaakt op: /Users/mohamedelharchaoui/Downloads/Tweedejaarsproject/excel-pdf-automaton/src/backend/temp_uploads/2e269c5c-ed91-4ee4-b676-a2345deda0b9/cao_samenvatting_2e269c5c-ed91-4ee4-b676-a2345deda0b9.xlsx
